In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Bidirectional, LSTM
from openpyxl.workbook import Workbook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set-up to have matplotlib use its support for notebook inline plots
%matplotlib inline

In [ ]:
btc_dfd = pd.read_csv('Binance_BTCUSDT_1h.csv')
btc_dfd

In [ ]:
btc_clean = btc_dfd.drop(['unix', 'symbol', 'Volume BTC', 'tradecount'], axis=1)
# res_btc = btc_clean[~(btc_clean['date'] < pd.to_datetime("2018-01-01").date())]
btc_clean = btc_clean[1:43016]
btc_clean

In [ ]:
btc = btc_clean.iloc[::-1]
btc

In [ ]:
btc = btc.reset_index(drop=True)
btc

In [ ]:
btc.duplicated().sum()

In [ ]:
duplicates = btc[btc.duplicated()]
print(duplicates)

In [ ]:
# No duplicated data

In [ ]:
btc.isnull().sum()

In [ ]:
# No missing value

In [ ]:
btc = btc[['open', 'high', 'low', 'Volume USDT', 'close']]
btc

In [ ]:
scaler = MinMaxScaler()
btc_df = scaler.fit_transform(np.array(btc))

In [ ]:
btc_df

In [ ]:
training_size = int(len(btc_df)*0.8)
test_size = len(btc_df)-training_size
train_data,test_data = btc_df[0:training_size,:],btc_df[training_size:len(btc_df),:]
print('train_data: ', train_data.shape)
print('test_data: ', test_data.shape)

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX = []
    dataY = []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step)]  
        b = dataset[i+time_step][4]
        dataX.append(a)
        dataY.append(float(b))
    
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 10
train_X, train_y = create_dataset(train_data, time_step)
test_X, test_y = create_dataset(test_data, time_step)
# reshape input to be [samples, time steps, features] which is required for LSTM
# X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
print("X_train: ", train_X.shape)
print("X_test: ", test_X.shape)

In [ ]:
train_X

In [ ]:
train_y

In [ ]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

## Model Training

In [ ]:
hyperparams = []
batch = [16, 32, 64]
epoch = [25, 50, 100]
neuron = [50, 60, 100]
for j in batch:
    for k in epoch:
        for l in neuron:
            hyperparams.append((j,k,l))
hyperparams

In [ ]:
hyperparam1 = hyperparams[:9]
hyperparam2 = hyperparams[9:18]
hyperparam3 = hyperparams[18:27]
hyperparam1

In [ ]:
hyperparam2

In [ ]:
hyperparam3

In [ ]:
%run Evaluation.ipynb

In [ ]:
bilstms1 = []
models1 = []
for batch, epoch, neuron in hyperparam1:
    model = Sequential()
    model.add(Bidirectional(LSTM(neuron, input_shape=(train_X.shape[1], train_X.shape[2]))))
    model.add(Dense(units=1))
    model.compile(loss='mse',optimizer='adam')
    bilstm = model.fit(train_X, train_y, epochs=epoch, batch_size=batch, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    bilstms1.append(bilstm)
    models1.append(model)

In [ ]:
i=0
wb1 = Workbook()
ws1 = wb1.active
for x in models1:
    # make a prediction
    test_x2 = test_X
    yhat = x.predict(test_x2)
    inv_yhat = np.concatenate((np.zeros((len(yhat), 4)),yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((np.zeros((len(yhat), 4)), test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close'])
    y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close'])
    print(hyperparam1[i])
    print("Epoch: "+ str(bilstms1[i].params['epochs']))
    print("Neurons: "+str(x.layers[0].layer.units))
    i = i+1
    ws1['A'+str(i)] = 'BiLSTM'
    ws1['B'+str(i)] = hyperparam1[i-1][0]
    ws1['C'+str(i)] = hyperparam1[i-1][1]
    ws1['D'+str(i)] = hyperparam1[i-1][2]
    print('RMSE')
    print(RMSE(y_df['Close'],yhat_df['Close']))
    ws1['E'+str(i)] = RMSE(y_df['Close'],yhat_df['Close'])
    print('MAE')
    print(MAE(y_df['Close'],yhat_df['Close']))
    ws1['F'+str(i)] = MAE(y_df['Close'],yhat_df['Close'])
    print('MAPE')
    print(MAPE(y_df['Close'],yhat_df['Close']))
    ws1['G'+str(i)] = MAPE(y_df['Close'],yhat_df['Close'])
wb1.save('BiLSTM_BTC_result1.xlsx')

In [ ]:
bilstms2 = []
models2 = []
for batch, epoch, neuron in hyperparam2:
    model = Sequential()
    model.add(Bidirectional(LSTM(neuron, input_shape=(train_X.shape[1], train_X.shape[2]))))
    model.add(Dense(units=1))
    model.compile(loss='mse',optimizer='adam')
    bilstm = model.fit(train_X, train_y, epochs=epoch, batch_size=batch, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    bilstms2.append(bilstm)
    models2.append(model)

In [ ]:
i=0
wb2 = Workbook()
ws2 = wb2.active
for x in models2:
    # make a prediction
    test_x2 = test_X
    yhat = x.predict(test_x2)
    inv_yhat = np.concatenate((np.zeros((len(yhat), 4)),yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((np.zeros((len(yhat), 4)), test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close'])
    y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close'])
    print(hyperparam2[i])
    print("Epoch: "+ str(bilstms2[i].params['epochs']))
    print("Neurons: "+str(x.layers[0].layer.units))
    i = i+1
    ws2['A'+str(i)] = 'BiLSTM'
    ws2['B'+str(i)] = hyperparam2[i-1][0]
    ws2['C'+str(i)] = hyperparam2[i-1][1]
    ws2['D'+str(i)] = hyperparam2[i-1][2]
    print('RMSE')
    print(RMSE(y_df['Close'],yhat_df['Close']))
    ws2['E'+str(i)] = RMSE(y_df['Close'],yhat_df['Close'])
    print('MAE')
    print(MAE(y_df['Close'],yhat_df['Close']))
    ws2['F'+str(i)] = MAE(y_df['Close'],yhat_df['Close'])
    print('MAPE')
    print(MAPE(y_df['Close'],yhat_df['Close']))
    ws2['G'+str(i)] = MAPE(y_df['Close'],yhat_df['Close'])
wb2.save('BiLSTM_BTC_result2.xlsx')

In [ ]:
bilstms3 = []
models3 = []
for batch, epoch, neuron in hyperparam3:
    model = Sequential()
    model.add(Bidirectional(LSTM(neuron, input_shape=(train_X.shape[1], train_X.shape[2]))))
    model.add(Dense(units=1))
    model.compile(loss='mse',optimizer='adam')
    bilstm = model.fit(train_X, train_y, epochs=epoch, batch_size=batch, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    bilstms3.append(bilstm)
    models3.append(model)

In [ ]:
i=0
wb3 = Workbook()
ws3 = wb3.active
for x in models3:
    # make a prediction
    test_x2 = test_X
    yhat = x.predict(test_x2)
    inv_yhat = np.concatenate((np.zeros((len(yhat), 4)),yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((np.zeros((len(yhat), 4)), test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close'])
    y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close'])
    print(hyperparam3[i])
    print("Epoch: "+ str(bilstms3[i].params['epochs']))
    print("Neurons: "+str(x.layers[0].layer.units))
    i = i+1
    ws3['A'+str(i)] = 'BiLSTM'
    ws3['B'+str(i)] = hyperparam3[i-1][0]
    ws3['C'+str(i)] = hyperparam3[i-1][1]
    ws3['D'+str(i)] = hyperparam3[i-1][2]
    print('RMSE')
    print(RMSE(y_df['Close'],yhat_df['Close']))
    ws3['E'+str(i)] = RMSE(y_df['Close'],yhat_df['Close'])
    print('MAE')
    print(MAE(y_df['Close'],yhat_df['Close']))
    ws3['F'+str(i)] = MAE(y_df['Close'],yhat_df['Close'])
    print('MAPE')
    print(MAPE(y_df['Close'],yhat_df['Close']))
    ws3['G'+str(i)] = MAPE(y_df['Close'],yhat_df['Close'])
wb3.save('BiLSTM_BTC_result3.xlsx')

In [ ]:
i=0
for x in models1:
    x.save('BiLSTM_BTC'+str(hyperparam1[i])+'.h5')
    i = i+1

In [ ]:
i=0
for x in models2:
    x.save('BiLSTM_BTC'+str(hyperparam2[i])+'.h5')
    i = i+1

In [ ]:
i=0
for x in models3:
    x.save('BiLSTM_BTC'+str(hyperparam3[i])+'.h5')
    i = i+1

In [ ]:
test_X